In [1]:
import ollama 
import os
from tqdm import tqdm
import json
import signal
import argparse
import wandb
import pandas as pd
import matplotlib.pyplot as plt
import copy
import numpy as np
import re

import sys
from collections import defaultdict

In [8]:
sys.argv = [
    'notebook',  
    '--modelname', 'llama3.2-vision:90b',
    '--data', 'gully',
    #'--data_path','/root/home/WACV-2025-Workshop-ViGIR/results/proposed/test_llama3_90b.json',
    '--data_path','/mnt/Gbenga_Enemy/ramy/results/test_Qwen2-VL-72B-Instruct-QAs.json',
    '--subset', 'train',
    '--results_dir', '/mnt/Gbenga_Enemy/ramy/results',
    '--timeout', '20',
    '--model_unloading'
    
]

In [9]:
parser = argparse.ArgumentParser(description="A script to evaluate V-LLMs on different image classification datasets")

parser.add_argument("--modelname", type=str, required=True, help="The name of the V-LLM model")
parser.add_argument("--data", type=str, required=True, help="Dataset name")
parser.add_argument("--data_path", type=str, required=True, help="Path to the image data dir")
parser.add_argument("--subset", type=str, required=True, help="train, test or validation set")
parser.add_argument("--results_dir", type=str, required=True, help="Folder name to save results")
parser.add_argument("--timeout", type=int, default=40, help="time out duration to skip one sample")
parser.add_argument("--model_unloading", action="store_true", help="Enables unloading mode. Every 100 samples it unloades the model from the GPU to avoid carshing.")

args = parser.parse_args()

In [10]:
# Load test set:
file_path = os.path.join(args.data_path)
with open(file_path, 'r') as file:
    data = json.load(file)

print('Number of Annotated GT Images: ', len(data.keys()))
data_keys_test = list(data.keys())
print('Number of Annotated GT Images (List): ', len(data_keys_test))

Number of Annotated GT Images:  311
Number of Annotated GT Images (List):  311


In [11]:
data

{'415': [[{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!',
   ['Yes.

In [14]:
data_reformated = {}

for key, item in data.items():
    q_and_a = []
    for i in range(len(item)):
        tmp = item[i][1:]   
        q_and_a.append(tmp)

    data_reformated[key] = q_and_a
    

In [15]:
data_reformated

{'415': [['Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!',
   ['Yes.']],
  ['Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!',
   ['Yes.']],
  ['Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!',
   ['Yes.']],
  ['Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!',
   ['Yes.']],
  ['Given these six images of the exact same area and collected over a period of 10 years, are there narrow and shallow channels which appear intermittently deeper or more indented into the soil? Ans

In [18]:
data_prompt = {}
for key, item in data_reformated.items():
    prompt = "\n".join([f"Q: {qa[0]}\nA: {qa[1][0]}" for qa in item])
    data_prompt[key] = [prompt]

In [19]:
data_prompt

{'415': ['Q: Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there narrow and shallow channels which appear intermittently deeper or more indented into the soil? Answer with yes or no only!\n

In [20]:
model_name = args.modelname
ollama.pull(model_name)

timeout_duration = args.timeout

options= {  # new
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048, # must be set, otherwise slightly random output
        }

model_labels = {}
count = 0

In [21]:
count = 0
saving_response=copy.deepcopy(data_prompt)

for key, info in tqdm(data_prompt.items()):

    #print(key)
    #print(info[0])
    #sys.exit()
    question = "Based on the following questions and their answers, determine if there is evidence of an ephemeral gully in the observed area. Carefully analyze all the questions and the responses to assess.\n\n"
    question += info[0]
    question += "\n\nAfter considering these responses, provide a clear conclusion: Is there evidence of an ephemeral gully? Answer with yes or no only."
    
    #print(question)
    #sys.exit()
        
    count+=1
    #for question in questions:
    response = ollama.generate(model=model_name, 
                               prompt=question, 
                               #images=info, 
                               options=options)
    saving_response[key].append(response['response'])
    print(response['response'])

    #sys.exit()
        

  0%|▎                                                                                | 1/311 [00:25<2:10:54, 25.34s/it]

Yes.


  1%|▌                                                                                  | 2/311 [00:26<58:37, 11.38s/it]

Yes.


  1%|▊                                                                                  | 3/311 [00:35<51:39, 10.06s/it]

...

Yes.


100%|████████████████████████████████████████████████████████████████████████████████▋| 310/311 [10:34<00:02,  2.39s/it]

Yes. 

(Note: The presence of a low point in the terrain, lack of vegetation suggesting an evolving channel, varying soil texture, and nearby human activity such as tillage or machinery tracks are indicative of potential ephemeral gully formation.)


100%|█████████████████████████████████████████████████████████████████████████████████| 311/311 [10:35<00:00,  2.04s/it]

Yes.


In [22]:
def extract_first_yes_no(text):
    # Use regular expressions to find "Yes" or "No" (case-insensitive)
    match = re.search(r'\b(Yes|No)\b', text, re.IGNORECASE)
    if match:
        # Return the first match in its original case
        return match.group(0)
    return None

In [23]:
saving_response_reformat = {}

for key, item in saving_response.items():
    #print(key)
    #print(item[-1])
    text = extract_first_yes_no(item[-1])

    print(f"Image: {key} - Answer: {text}")
    
    if text == 'Yes':
        saving_response_reformat[key] = ["4"]

    if text == 'No':
        saving_response_reformat[key] = ["0"]
        

Image: 415 - Answer: Yes
Image: 1020 - Answer: Yes
Image: 105 - Answer: Yes
Image: 439 - Answer: Yes
Image: 914 - Answer: Yes
Image: 1099 - Answer: Yes
Image: 1065 - Answer: Yes
Image: 373 - Answer: Yes
Image: 166 - Answer: Yes
Image: 396 - Answer: Yes
Image: 837 - Answer: Yes
Image: 685 - Answer: Yes
Image: 226 - Answer: Yes
Image: 956 - Answer: Yes
Image: 70 - Answer: Yes
Image: 604 - Answer: Yes
Image: 1067 - Answer: Yes
Image: 118 - Answer: Yes
Image: 774 - Answer: Yes
Image: 521 - Answer: Yes
Image: 910 - Answer: Yes
Image: 975 - Answer: Yes
Image: 352 - Answer: Yes
Image: 761 - Answer: Yes
Image: 1007 - Answer: Yes
Image: 428 - Answer: Yes
Image: 1038 - Answer: Yes
Image: 50 - Answer: Yes
Image: 838 - Answer: Yes
Image: 126 - Answer: Yes
Image: 1078 - Answer: Yes
Image: 944 - Answer: Yes
Image: 532 - Answer: Yes
Image: 1041 - Answer: Yes
Image: 540 - Answer: Yes
Image: 128 - Answer: Yes
Image: 722 - Answer: Yes
Image: 478 - Answer: Yes
Image: 639 - Answer: Yes
Image: 668 - Answer

In [24]:
saving_response_reformat

{'415': ['4'],
 '1020': ['4'],
 '105': ['4'],
 '439': ['4'],
 '914': ['4'],
 '1099': ['4'],
 '1065': ['4'],
 '373': ['4'],
 '166': ['4'],
 '396': ['4'],
 '837': ['4'],
 '685': ['4'],
 '226': ['4'],
 '956': ['4'],
 '70': ['4'],
 '604': ['4'],
 '1067': ['4'],
 '118': ['4'],
 '774': ['4'],
 '521': ['4'],
 '910': ['4'],
 '975': ['4'],
 '352': ['4'],
 '761': ['4'],
 '1007': ['4'],
 '428': ['4'],
 '1038': ['4'],
 '50': ['4'],
 '838': ['4'],
 '126': ['4'],
 '1078': ['4'],
 '944': ['4'],
 '532': ['4'],
 '1041': ['4'],
 '540': ['4'],
 '128': ['4'],
 '722': ['4'],
 '478': ['4'],
 '639': ['4'],
 '668': ['4'],
 '343': ['4'],
 '1021': ['4'],
 '552': ['4'],
 '848': ['4'],
 '74': ['4'],
 '520': ['4'],
 '1032': ['4'],
 '615': ['4'],
 '536': ['4'],
 '1117': ['4'],
 '646': ['4'],
 '390': ['4'],
 '923': ['4'],
 '194': ['4'],
 '216': ['4'],
 '99': ['4'],
 '372': ['4'],
 '857': ['4'],
 '335': ['4'],
 '505': ['4'],
 '972': ['4'],
 '727': ['4'],
 '1064': ['4'],
 '740': ['4'],
 '182': ['4'],
 '316': ['4'],
 '

In [100]:
# # Save the dictionary as a JSON file
# filename =  'results_'+args.modelname+'.json'
# with open(os.path.join(args.results_dir, filename), 'w') as json_file:
#     json.dump(saving_response_reformat, json_file, indent=4)

# print(f"JSON file {filename} has been created.")

JSON file results_llama3.2-vision:90b.json has been created.


In [25]:
# Save the dictionary as a JSON file
filename =  'QwenVQA_LlamaLLM.json'
with open(os.path.join(args.results_dir, filename), 'w') as json_file:
    json.dump(saving_response_reformat, json_file, indent=4)

print(f"JSON file {filename} has been created.")

JSON file QwenVQA_LlamaLLM.json has been created.


In [28]:
valid_file=os.path.join('/mnt/Gbenga_Enemy/ramy/WACV-2025-Workshop-ViGIR/Validation_Set.json')
test_file=os.path.join('/mnt/Gbenga_Enemy/ramy/WACV-2025-Workshop-ViGIR/Testing_Set.json')

with open(valid_file, 'r') as file:
    valid = json.load(file) 

with open(test_file, 'r') as file:
    test = json.load(file) 